In [1]:
import logging

logging.basicConfig(level=logging.INFO)

class AttrLoggingMeta(type):

    def __new__(cls, name, bases, attrs):
        
        for key, attr in attrs.items():
            if callable(attr) and key != '__init__':
                attrs[key] = AttrLoggingMeta.log_access(attr)

        return super().__new__(cls, name, bases, attrs)     
    
    def __init__(cls, name, bases, attrs, **extra_kwargs): 
        cls.__setattr__ = AttrLoggingMeta.log_write(cls.__setattr__)
        cls.__getattribute__ = AttrLoggingMeta.log_read(cls.__getattribute__)

        super().__init__(cls, bases, attrs, **extra_kwargs)   


    @staticmethod
    def log_access(f):
        def wrapper(*args, **kwargs):
            logging.info(f'Calling method {f.__name__}')

            return f(*args, **kwargs)
        
        return wrapper
    @staticmethod
    def log_read(f):
        def wrapper(*args, **kwargs):

            if args[1] in ('__getattribute__', '__dict__'):
                return object.__getattribute__(*args)
            
            if not callable(object.__getattribute__(*args)):
                logging.info(f'Reading attribute {args[1]}')

            return f(*args, **kwargs)
 
        return wrapper 
    
    @staticmethod
    def log_write(f):
        def wrapper(*args, **kwargs):
            logging.info(f'Writing attribute {args[1]}')

            return f(*args, **kwargs)
        
        return wrapper  

    def __call__(cls, *args, **kwargs):  
        return super().__call__(*args, **kwargs)


class LoggedClass(metaclass = AttrLoggingMeta):

    def meth(self):
        return 'method 1'

    def meth_2(self):
        return 'method 2'


a = LoggedClass()
a.test = 1
print(a.test)
print(a.meth())


INFO:root:Writing attribute test
INFO:root:Reading attribute test
INFO:root:Calling method meth


1
method 1


In [2]:
def create_class_with_methods(name: str, attributes: dict, methods: dict):
    
    return type(
    name,
    (),  
    {  
        '__doc__': 'This is demo of dynamic Class',  
        **attributes,
        **methods
    }  
)  



class_attributes = {'species':'Human', 'age':25}
class_methods = {'greet': lambda self: f"Hello, I am a {self.species} and I am {self.age} years old."}
DynamicClass = create_class_with_methods('DynamicClass', class_attributes, class_methods )

instance = DynamicClass()
print(instance.greet())

Hello, I am a Human and I am 25 years old.


In [ ]:
from types import FunctionType 
from typing import List

function_name = 'complex_function'
parameters = ['x','y']
function_body = """
    if x > y:
        return x-y
    else:
        return y-x
"""

def generate_complex_function(name:str, args:List, code:str):
    '''function for generate function from source code'''
    func_code = compile(f'def f({", ".join(args)}):' + code, "<string>", "exec")

    return FunctionType(func_code.co_consts[0], globals(), name)


complex_func = generate_complex_function(function_name, parameters, function_body)

print(complex_func.__name__)

print(complex_func(10, 5))
print(complex_func(5, 10))

complex_function
5
5
